# DHS grouping
Does further preprocessing steps after the unified columns have been loaded from DHS_loading_from_preprocessing.ipynb
Especially:
- loading locations file and merging them
- groups on either GEID (a survey ID) and adm2 or GEID and cluster ID
- creates DHS dataset

In [20]:
from icecream import ic
import scipy.stats
import pandas as pd
import numpy as np
from warnings import warn

In [21]:
# folder_path = "/mnt/datadisk/data/surveys/DHS_final_raw_data/"
# work_dir = "/mnt/datadisk/data/Projects/water/pickles/"
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
# min_version = 3
overwrite_pqt = True
dataset_type = 'HR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 25 surveys
# 'BR' (birth R), 'HW!' (WHO height/weight standards children), 'IR'/'PR'? (Individual R, hh member R,), 'KR' (children R), 'AR' (HIV R)

# 'adm2_gaul' or 'cluster number' (possibly 'adm1_gaul' etc.); additionally to survey ID
group_by_col = 'adm2_gaul'
preprocessing_f = f"{input_dir}Manually_adjusted_df_V3_{dataset_type}.pkl"
# unify_columns_f = f"{input_dir}summary_df_V3_{dataset_type}_unify_cols_reviewed.csv"
locations_f = input_dir + 'locations.csv'
add_str = '_w_NANs' # '_w_NANs' or '_wo_NANs'
# numeric_out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}_numeric{add_str}.pkl"
out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}{add_str}.pkl"
meta_data_cols_out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}_meta_data_cols{add_str}.pkl"
# cut_perc = 10

In [22]:
locations_df = pd.read_csv(locations_f)
print(locations_df.columns.to_list())
locations_df = locations_df[['GEID', 'TIF_name', 'adm2_name', 'adm1_name', 'adm0_name', 'DHSCLUST', 'URBAN_RURA']]
locations_df = locations_df.rename(columns={'GEID': 'GEID_init'})
locations_df = locations_df.rename(columns={'TIF_name': 'GEID', 'adm2_name': 'adm2_gaul', 'adm1_name': 'adm1_gaul', 'adm0_name': 'adm0_gaul', 'DHSCLUST': 'DHSCLUST'})
locations_df

['country', 'DHSID', 'DHSCC', 'DHSYEAR', 'DHSCLUST', 'CCFIPS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1SALBNA', 'ADM1SALBCO', 'ADM1DHS', 'ADM1NAME', 'DHSREGCO', 'DHSREGNA', 'SOURCE', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'ALT_GPS', 'ALT_DEM', 'DATUM', 'GEID', 'TIF_name', 'adm2_code', 'adm2_name', 'adm1_code', 'adm1_name', 'adm0_code', 'adm0_name', 'Shape_Area']


,GEID_init,GEID,adm2_gaul,adm1_gaul,adm0_gaul,DHSCLUST,URBAN_RURA
0,AOGE52FL,AOGE52FL00000001,Lobito,Benguela,Angola,1,U
1,AOGE52FL,AOGE52FL00000002,Lobito,Benguela,Angola,2,U
2,AOGE52FL,AOGE52FL00000003,Benguela,Benguela,Angola,3,U
3,AOGE52FL,AOGE52FL00000004,Sumbe,Cuanza Sul,Angola,4,U
4,AOGE52FL,AOGE52FL00000005,Dande,Bengo,Angola,5,U
...,...,...,...,...,...,...,...
70751,ZWGE72FL,ZWGE72FL00000396,Harare,Harare,Zimbabwe,396,U
70752,ZWGE72FL,ZWGE72FL00000397,Makoni,Manicaland,Zimbabwe,397,R
70753,ZWGE72FL,ZWGE72FL00000398,Hurungwe,Mashonaland West,Zimbabwe,398,R
70754,ZWGE72FL,ZWGE72FL00000399,Harare,Harare,Zimbabwe,399,U


In [23]:
df = pd.read_pickle(preprocessing_f)
print(len(df['generated clnr from case id'].dropna()), len(df['generated hhnr from case id'].dropna()))
#these are empty from steps before
df = df.drop(columns=['generated clnr from case id', 'generated hhnr from case id'])

12110 84200


In [24]:
df = df.merge(locations_df, on='GEID', how='inner')
df[df['cluster number'] != df['DHSCLUST']][['GEID', 'adm2_gaul', 'adm1_gaul', 'adm0_gaul', 'DHSCLUST', 'cluster number']]

,GEID,adm2_gaul,adm1_gaul,adm0_gaul,DHSCLUST,cluster number
455349,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455350,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455351,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455352,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455353,GHGE33FL00000082,Agona West,Central,Ghana,81,82
...,...,...,...,...,...,...
1479154,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132
1479155,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132
1479156,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132
1479157,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132


In [25]:
df[['type of place of residence', 'URBAN_RURA']].value_counts(normalize=True, dropna=False) * 100


type of place of residence  URBAN_RURA
rural                       R             65.062716
urban                       U             34.874341
rural                       U              0.050168
urban                       R              0.012775
Name: proportion, dtype: float64

In [26]:
df2 = df[~df['cluster number'] != df['DHSCLUST']]
print(len(df))
df2

1612581


,country code and phase,GEID,DHSID,DHSID + HHID,version_nr,subversion_nr,cluster number,household number,number of household members,month of interview,...,share toilet with other households: applicable,share toilet with other households: not applicable,number of households sharing toilet: applicable,number of households sharing toilet: not applicable,GEID_init,adm2_gaul,adm1_gaul,adm0_gaul,DHSCLUST,URBAN_RURA
0,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000001,5,1,1,1,3.0,12.0,...,1.0,0.0,1.0,0.0,AOGE52FL,Lobito,Benguela,Angola,1,U
1,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000002,5,1,1,2,3.0,12.0,...,1.0,0.0,1.0,0.0,AOGE52FL,Lobito,Benguela,Angola,1,U
2,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000003,5,1,1,3,3.0,12.0,...,1.0,0.0,1.0,0.0,AOGE52FL,Lobito,Benguela,Angola,1,U
3,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000004,5,1,1,4,5.0,12.0,...,1.0,0.0,1.0,0.0,AOGE52FL,Lobito,Benguela,Angola,1,U
4,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000005,5,1,1,5,3.0,12.0,...,1.0,0.0,1.0,0.0,AOGE52FL,Lobito,Benguela,Angola,1,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612576,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000025,7,2,400,25,5.0,11.0,...,1.0,0.0,1.0,0.0,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R
1612577,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000026,7,2,400,26,2.0,11.0,...,0.0,1.0,0.0,1.0,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R
1612578,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000027,7,2,400,27,5.0,11.0,...,1.0,0.0,1.0,0.0,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R
1612579,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000028,7,2,400,28,8.0,11.0,...,1.0,0.0,1.0,0.0,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R


In [27]:
### Splitting numeric and string columns as well as meta data columns
df = df2

df = df[(df['year of interview'].notna())
        & (df['month of interview'].notna()) & (df['day of interview'].notna())]
# Initialize empty lists to hold the names of numeric and categorical columns
numeric_columns = [df[['GEID', 'household number', 'GEID_init', group_by_col]]]
categorical_columns = [df[['GEID', 'household number', 'GEID_init', group_by_col]]]
meta_data_cols = ['country code and phase', 'GEID', 'DHSID', 'DHSID + HHID', 'version_nr', 'subversion_nr', 'household number', 'case identification', 'DHSCLUST', 'country code', 
       'adm1_gaul', 'adm0_gaul', 'GEID_init', 'month of interview',
       'year of interview', 'day of interview', 'cluster number', 'adm2_gaul', 'URBAN_RURA'
    #    'generated hhnr from case id',
    #    'generated clnr from case id', 
       ]
if group_by_col not in meta_data_cols:
    warn(f'group_by_col {group_by_col} not in meta_data_cols, may result in errors!')
    meta_data_cols.append(group_by_col)
    
# Iterate over each column
for col in df.columns:
    if col in meta_data_cols:
        continue
    found_nr = False
    # Create a new column for the numeric values
    nrcol = pd.to_numeric(df[col], errors='coerce')
    if nrcol.notna().sum() > 0:
        found_nr = True
        numeric_columns.append(nrcol)
    # Create a new column for the non-numeric values
    str_col = df[col].where(nrcol.isna())
    if str_col.notna().sum() > 0:
        categorical_columns.append(str_col)
        if found_nr:
            print('found both', col)


found both language of interview
found both type of cooking fuel
found both owns land usable for agriculture
found both hectares of agricultural land (1 decimal)
found both number of rooms used for sleeping


In [28]:
meta_data_df = df[meta_data_cols].copy()
print('cluster number' in meta_data_df.columns, 'GEID_init' in meta_data_df.columns)
print([c for c in meta_data_df.columns if c in ['cluster number', 'GEID_init']])
# meta_data_df['month of interview'] = meta_data_df['month of interview'].fillna(6)
# meta_data_df['day of interview'] = meta_data_df['day of interview'].fillna(15)

meta_data_df['year'] = meta_data_df['year of interview'].astype(float).astype('Int32')
meta_data_df['month'] = meta_data_df['month of interview'].astype(float).astype('Int32')
meta_data_df['day'] = meta_data_df['day of interview'].astype(float).astype('Int32')

#replace days >28 with 28
meta_data_df.loc[meta_data_df['day'] > 28, 'day'] = 28

# Now you can combine the year, month, and day columns into a single date column
meta_data_df['date'] = pd.to_datetime(meta_data_df[['year', 'month', 'day']])

# Group the DataFrame by 'GEID_init' and 'adm2_gaul'/'cluster number'
grouped = meta_data_df.groupby(['GEID_init', group_by_col])

# Calculate the median date for each group and reset the index
median_date = grouped['date'].median().reset_index()

# Rename the date column in median_date
median_date.rename(columns={'date': 'median_date'}, inplace=True)

# Merge meta_data_df with median_date
meta_data_df = pd.merge(meta_data_df, median_date, on=['GEID_init', group_by_col], how='left')

# Convert the median date to separate year, month, and day columns
meta_data_df['year'] = meta_data_df['median_date'].dt.year.astype('Int32')
meta_data_df['month'] = meta_data_df['median_date'].dt.month.astype('Int32')

meta_data_df[['year', 'month', 'year of interview', 'month of interview', 'day of interview', 'date', 'median_date']]


True True
['GEID_init', 'cluster number']


,year,month,year of interview,month of interview,day of interview,date,median_date
0,2006,12,2006.0,12.0,2.0,2006-12-02,2006-12-01
1,2006,12,2006.0,12.0,2.0,2006-12-02,2006-12-01
2,2006,12,2006.0,12.0,2.0,2006-12-02,2006-12-01
3,2006,12,2006.0,12.0,1.0,2006-12-01,2006-12-01
4,2006,12,2006.0,12.0,1.0,2006-12-01,2006-12-01
...,...,...,...,...,...,...,...
1605431,2015,10,2015.0,11.0,10.0,2015-11-10,2015-10-27
1605432,2015,10,2015.0,11.0,12.0,2015-11-12,2015-10-27
1605433,2015,10,2015.0,11.0,12.0,2015-11-12,2015-10-27
1605434,2015,10,2015.0,11.0,10.0,2015-11-10,2015-10-27


In [29]:
# Drop the original date column
meta_data_df['date'] = meta_data_df['median_date']
meta_data_df.drop(columns=['day', 'year of interview', 'month of interview', 'day of interview', 'median_date'], inplace=True)

# Create numeric df
numeric_df = pd.concat(numeric_columns, axis=1)

#drop cols which shouldnt be here 
numeric_df = numeric_df.drop(columns=['type of cooking fuel', 'language of interview', 'owns land usable for agriculture'])

# Create categorical df
categorical_df = pd.concat(categorical_columns, axis=1)

categorical_df = categorical_df.drop(columns=['number of rooms used for sleeping', 'hectares of agricultural land (1 decimal)'])
print(categorical_df.columns)

Index(['GEID', 'household number', 'GEID_init', 'adm2_gaul',
       'type of place of residence', 'source of drinking water',
       'has electricity', 'has radio', 'has bicycle', 'has car/truck',
       'region', 'type of toilet facility', 'has refrigerator',
       'translator used', 'has telephone (land-line)', 'has bank account',
       'language of questionnaire', 'share toilet with other households',
       'location of source for water',
       'anything done to water to make safe to drink',
       'water usually treated by: boil',
       'water usually treated by: add bleach/chlorine',
       'native language of respondent',
       'household selected for domestic violence module',
       'place where household members wash their hands',
       'presence of water at hand washing place',
       'items present: soap or detergent', 'items present: ash, mud, sand',
       'items present: none', 'location of toilet facility',
       'water not available for at least a day last two w

In [30]:
for col in categorical_df.columns:
    print(categorical_df[col].dtype)
    print(categorical_df[col].value_counts(normalize=True, dropna=False) * 100)
    print('\n')

object
GEID
EGGE32FL00670122    0.007350
EGGE42FL00850329    0.007163
EGGE61FL08400151    0.006727
EGGE42FL00550211    0.005481
EGGE61FL08400116    0.005419
                      ...   
EGGE4BFL00990102    0.000062
EGGE4BFL04130101    0.000062
EGGE61FL01234204    0.000062
NIGE81FL00000009    0.000062
EGGE4BFL00962401    0.000062
Name: proportion, Length: 67316, dtype: float64


int64
household number
1        1.990986
3        1.952180
5        1.931376
7        1.930317
8        1.916053
           ...   
18625    0.000062
19526    0.000062
10517    0.000062
15122    0.000062
6945     0.000062
Name: proportion, Length: 14803, dtype: float64


object
GEID_init
NGGE7BFL    2.505238
NGGE6AFL    2.380350
KEGE8AFL    2.361415
KEGE71FL    2.256334
NGGE52FL    2.122165
              ...   
MWGE71FL    0.212092
MWGE6AFL    0.212030
AOGE52FL    0.161887
CIGE3BFL    0.132176
MLGE63FL    0.098291
Name: proportion, Length: 155, dtype: float64


object
adm2_gaul
Bamako                             

In [31]:
for col in numeric_df.columns:
    print(numeric_df[col].dtype)
    print(numeric_df[col].value_counts(normalize=True, dropna=False) * 100)
    print('\n')

object
GEID
EGGE32FL00670122    0.007350
EGGE42FL00850329    0.007163
EGGE61FL08400151    0.006727
EGGE42FL00550211    0.005481
EGGE61FL08400116    0.005419
                      ...   
EGGE4BFL00990102    0.000062
EGGE4BFL04130101    0.000062
EGGE61FL01234204    0.000062
NIGE81FL00000009    0.000062
EGGE4BFL00962401    0.000062
Name: proportion, Length: 67316, dtype: float64


int64
household number
1        1.990986
3        1.952180
5        1.931376
7        1.930317
8        1.916053
           ...   
18625    0.000062
19526    0.000062
10517    0.000062
15122    0.000062
6945     0.000062
Name: proportion, Length: 14803, dtype: float64


object
GEID_init
NGGE7BFL    2.505238
NGGE6AFL    2.380350
KEGE8AFL    2.361415
KEGE71FL    2.256334
NGGE52FL    2.122165
              ...   
MWGE71FL    0.212092
MWGE6AFL    0.212030
AOGE52FL    0.161887
CIGE3BFL    0.132176
MLGE63FL    0.098291
Name: proportion, Length: 155, dtype: float64


object
adm2_gaul
Bamako                             

In [32]:
for col in meta_data_df.columns:
    print(meta_data_df[col].dtype)
    print(meta_data_df[col].value_counts(normalize=True, dropna=False) * 100)
    print('\n')
meta_data_df.to_csv(meta_data_cols_out_f.replace('.pkl', '.csv'), index=False)
meta_data_df.to_pickle(meta_data_cols_out_f)

object
country code and phase
NG6    3.224046
EG4    2.914660
KE6    2.660025
NG7    2.505238
KE8    2.361415
         ...   
SZ5    0.296244
NI7    0.294562
BJ3    0.280235
KM6    0.267591
TZ3    0.221871
Name: proportion, Length: 117, dtype: float64


object
GEID
EGGE32FL00670122    0.007350
EGGE42FL00850329    0.007163
EGGE61FL08400151    0.006727
EGGE42FL00550211    0.005481
EGGE61FL08400116    0.005419
                      ...   
EGGE4BFL00990102    0.000062
EGGE4BFL04130101    0.000062
EGGE61FL01234204    0.000062
NIGE81FL00000009    0.000062
EGGE4BFL00962401    0.000062
Name: proportion, Length: 67316, dtype: float64


object


DHSID
EGHR33SV00670122    0.007350
EGHR42SV00850329    0.007163
EGHR61SV08400151    0.006727
EGHR42SV00550211    0.005481
EGHR61SV08400116    0.005419
                      ...   
EGHR4ASV00990102    0.000062
EGHR4ASV04130101    0.000062
EGHR61SV01234204    0.000062
NIHR82SV00000009    0.000062
EGHR4ASV00962401    0.000062
Name: proportion, Length: 67316, dtype: float64


object
DHSID + HHID
AOHR51SV0000000100000001    0.000062
NGHR71SV0000009000000087    0.000062
NGHR71SV0000009100000041    0.000062
NGHR71SV0000009100000039    0.000062
NGHR71SV0000009100000036    0.000062
                              ...   
GNHR71SV0000017300000094    0.000062
GNHR71SV0000017300000089    0.000062
GNHR71SV0000017300000085    0.000062
GNHR71SV0000017300000080    0.000062
ZWHR72SV0000040000000029    0.000062
Name: proportion, Length: 1605436, dtype: float64


int64
version_nr
6    31.103638
7    26.287999
5    16.533764
4    13.969227
8     6.315481
3     5.130943
2     0.658949
Name: proportion, dtype:

In [33]:
print(len(categorical_df), len(df), len(numeric_df), len(meta_data_df))
print('GEID' in categorical_df.columns, 'GEID' in numeric_df.columns, 'GEID' in meta_data_df.columns)
print('household number' in categorical_df.columns, 'household number' in numeric_df.columns, 'household number' in meta_data_df.columns)

1605436 1605436 1605436 1605436
True True True
True True True


In [34]:
cols = ['GEID', 'adm2_gaul', 'adm1_gaul', 'adm0_gaul', 'year', 'month', 'household number', 'cluster number', 'date']
if group_by_col in cols:    
    cols.remove(group_by_col)
    
numeric_df2 = pd.merge(meta_data_df[cols], numeric_df, on=['GEID', 'household number'])
# categorical_df2 = pd.merge(meta_data_df[['GEID', 'household number']], categorical_df, on=['GEID', 'household number'])
assert len(df) == len(numeric_df2) == len(meta_data_df)
for na, df in [('numeric', numeric_df2), ('cat', categorical_df)]:
    for col in df.columns:
        if '_x' in col:
            print(na, col)
numeric_df2

,GEID,adm1_gaul,adm0_gaul,year,month,household number,cluster number,date,GEID_init,adm2_gaul,...,number of rooms used for sleeping,owns goats,owns horses/ donkeys/ mules,time to get to water source (minutes),owns cows/ bulls,owns chickens/poultry,share toilet with other households: applicable,share toilet with other households: not applicable,number of households sharing toilet: applicable,number of households sharing toilet: not applicable
0,AOGE52FL00000001,Benguela,Angola,2006,12,1,1,2006-12-01,AOGE52FL,Lobito,...,NaN,NaN,NaN,0.0,NaN,NaN,1.0,0.0,1.0,0.0
1,AOGE52FL00000001,Benguela,Angola,2006,12,2,1,2006-12-01,AOGE52FL,Lobito,...,NaN,NaN,NaN,0.0,NaN,NaN,1.0,0.0,1.0,0.0
2,AOGE52FL00000001,Benguela,Angola,2006,12,3,1,2006-12-01,AOGE52FL,Lobito,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.0,1.0,0.0
3,AOGE52FL00000001,Benguela,Angola,2006,12,4,1,2006-12-01,AOGE52FL,Lobito,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.0,1.0,0.0
4,AOGE52FL00000001,Benguela,Angola,2006,12,5,1,2006-12-01,AOGE52FL,Lobito,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605431,ZWGE72FL00000400,Mashonaland East,Zimbabwe,2015,10,25,400,2015-10-27,ZWGE72FL,Murehwa,...,2.0,7.0,0.0,0.0,NaN,31.0,1.0,0.0,1.0,0.0
1605432,ZWGE72FL00000400,Mashonaland East,Zimbabwe,2015,10,26,400,2015-10-27,ZWGE72FL,Murehwa,...,2.0,0.0,0.0,0.0,NaN,10.0,0.0,1.0,0.0,1.0
1605433,ZWGE72FL00000400,Mashonaland East,Zimbabwe,2015,10,27,400,2015-10-27,ZWGE72FL,Murehwa,...,3.0,3.0,0.0,10.0,NaN,0.0,1.0,0.0,1.0,0.0
1605434,ZWGE72FL00000400,Mashonaland East,Zimbabwe,2015,10,28,400,2015-10-27,ZWGE72FL,Murehwa,...,3.0,8.0,0.0,0.0,NaN,5.0,1.0,0.0,1.0,0.0


In [35]:
# List of columns to exclude from the aggregation
cols_to_exclude = ['adm1_gaul', 'adm0_gaul', 'year', 'month', 'GEID_init', 'adm2_gaul', 'GEID', 'household number', 'cluster number']
# if 'cluster number' != group_by_col:
#     cols_to_exclude.append('cluster number')
cols_to_include_again = ['adm1_gaul', 'adm0_gaul', 'year', 'month', 'date']
if 'adm2_gaul' != group_by_col:
    cols_to_include_again.append('adm2_gaul')
# List of columns to include in the aggregation
cols_to_include = [col for col in numeric_df.columns if col not in cols_to_exclude]

numeric_df = numeric_df2
# Group the DataFrame
grouped_df = numeric_df.groupby(['GEID_init', group_by_col])

# Perform the aggregation on a copy of the DataFrame
# agg_df = grouped_df[cols_to_include].agg(['mean', 'median', 'std', lambda x: scipy.stats.skew(x.dropna()) if not x.dropna().empty else np.nan])
agg_df = grouped_df[cols_to_include].agg(['mean', 'median', 'std', 
                                          lambda x: scipy.stats.skew(x.dropna()) if not x.dropna().empty else np.nan,
                                          lambda x: scipy.stats.kurtosis(x.dropna()) if not x.dropna().empty else np.nan])
# Get the first occurrence of the cols_to_exclude in each group
grouped_df = grouped_df[cols_to_include_again].agg('first')
# grouped_df = grouped_df[cols_to_exclude].first()
# Add a second level to the columns in grouped_df
grouped_df.columns = pd.MultiIndex.from_product([grouped_df.columns, ['']])

assert len(grouped_df) == len(agg_df)
# Merge the aggregation result back to the original DataFrame
grouped_df = pd.concat([grouped_df, agg_df], axis=1)

# Rename the last aggregation function to 'skew'
# Rename the last two aggregation functions to 'skewness' and 'kurtosis'
grouped_df.rename(columns={'<lambda_0>': 'skewness', '<lambda_1>': 'kurtosis'}, level=1, inplace=True)
# grouped_df.rename(columns={'<lambda_0>': 'skewness'}, level=1, inplace=True)

# Calculate the count of non-NaN values in each column
count_df = numeric_df.groupby(['GEID_init', group_by_col]).count()

# Calculate the size of each group
households = numeric_df.groupby(['GEID_init', group_by_col]).size()

assert len(grouped_df) == len(households)

# Add the size of each group to the grouped DataFrame
grouped_df[('amount of households', '')] = households.values

# Calculate the percentage of valid answers
percentage_df = count_df.div(grouped_df[('amount of households', '')], axis=0) * 100

assert len(percentage_df) == len(grouped_df)
# Add the percentage of valid answers to the grouped DataFrame
for col in percentage_df.columns:
    grouped_df[(col, 'percentage of valid answers')] = percentage_df[col]

# Flatten the MultiIndex
grouped_df.columns = [': '.join(col).strip() if isinstance(col, tuple) and col[1] else col[0] for col in grouped_df.columns.values]

# Reset the index
grouped_df.reset_index(inplace=True)
numerical_grouped_df = grouped_df
grouped_df

,GEID_init,adm2_gaul,adm1_gaul,adm0_gaul,year,month,date,number of household members: mean,number of household members: median,number of household members: std,...,number of rooms used for sleeping: percentage of valid answers,owns goats: percentage of valid answers,owns horses/ donkeys/ mules: percentage of valid answers,time to get to water source (minutes): percentage of valid answers,owns cows/ bulls: percentage of valid answers,owns chickens/poultry: percentage of valid answers,share toilet with other households: applicable: percentage of valid answers,share toilet with other households: not applicable: percentage of valid answers,number of households sharing toilet: applicable: percentage of valid answers,number of households sharing toilet: not applicable: percentage of valid answers
0,AOGE52FL,Ambaca,Kuanza Norte,Angola,2007,3,2007-03-06,5.160000,5.0,1.907878,...,0.0,0.0,0.0,52.000000,0.000000,0.000000,100.0,100.0,100.0,100.0
1,AOGE52FL,Amboim,Cuanza Sul,Angola,2006,12,2006-12-11,5.280000,5.0,2.282542,...,0.0,0.0,0.0,100.000000,0.000000,0.000000,100.0,100.0,100.0,100.0
2,AOGE52FL,Andulo,Bie,Angola,2007,1,2007-01-08,4.000000,4.0,1.632993,...,0.0,0.0,0.0,80.000000,0.000000,0.000000,100.0,100.0,100.0,100.0
3,AOGE52FL,Baia Farta,Benguela,Angola,2006,11,2006-11-27,5.208333,5.0,2.283764,...,0.0,0.0,0.0,33.333333,0.000000,0.000000,100.0,100.0,100.0,100.0
4,AOGE52FL,Balombo,Benguela,Angola,2006,12,2006-12-07,4.210526,4.0,1.618605,...,0.0,0.0,0.0,100.000000,0.000000,0.000000,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15247,ZWGE72FL,Umzingwane,Matabeleland South,Zimbabwe,2015,7,2015-07-10,4.642857,4.0,2.865006,...,100.0,100.0,100.0,100.000000,19.047619,100.000000,100.0,100.0,100.0,100.0
15248,ZWGE72FL,Uzumba-Maramba-Pfungwe,Mashonaland East,Zimbabwe,2015,10,2015-10-12,4.075949,4.0,2.080262,...,100.0,100.0,100.0,100.000000,15.189873,100.000000,100.0,100.0,100.0,100.0
15249,ZWGE72FL,Zaka,Masvingo,Zimbabwe,2015,10,2015-10-19,4.366337,4.0,2.415462,...,100.0,100.0,100.0,99.009901,6.930693,100.000000,100.0,100.0,100.0,100.0
15250,ZWGE72FL,Zvimba,Mashonaland West,Zimbabwe,2015,11,2015-11-25,4.546154,4.0,2.385718,...,100.0,100.0,100.0,100.000000,36.153846,100.000000,100.0,100.0,100.0,100.0


In [36]:
#continue here
del grouped_df
grouped_dfs = []

# categorical_df = categorical_df2
# Group the DataFrame by 'GEID_init' and 'adm2_gaul'
grouped = categorical_df.groupby(['GEID_init', group_by_col])

for col in categorical_df.columns:
    if col not in cols_to_exclude:
        # Calculate the normalized count of each category excluding NaNs
        counts = grouped[col].value_counts(normalize=True).unstack()
        
        # Only fill NaNs with 0 if there were values in the group
        if not counts.empty:
            counts = counts.fillna(0)
        
        if add_str == '_w_NANs':
            # Calculate the normalized count of NaNs
            nan_counts = grouped[col].apply(lambda x: x.isna().mean()).rename('NaN')
            
            # Append the count of NaNs to the counts of the other categories
            counts = pd.concat([counts, nan_counts], axis=1)
        
        # Flatten the MultiIndex and add a ': ' between the original column name and the value_counts.index items
        counts.columns = [f'{col}: {c}' for c in counts.columns]
        
        grouped_dfs.append(counts)
        
# Concatenate the results
grouped_df = pd.concat(grouped_dfs, axis=1)
grouped_df.reset_index(inplace=True)
categorical_grouped_df = grouped_df[[c for c in grouped_df.columns if c not in numerical_grouped_df.columns or c in ['GEID_init', group_by_col]]]

/tmp/ipykernel_1220311/4102915504.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_df.reset_index(inplace=True)
/tmp/ipykernel_1220311/4102915504.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_df.reset_index(inplace=True)


In [37]:
# merge both dfs
grouped_df = pd.merge(numerical_grouped_df, categorical_grouped_df, on=['GEID_init', group_by_col], how='inner')
grouped_df

,GEID_init,adm2_gaul,adm1_gaul,adm0_gaul,year,month,date,number of household members: mean,number of household members: median,number of household members: std,...,has television: NaN,has motorcycle/scooter: no,has motorcycle/scooter: yes,has motorcycle/scooter: NaN,water usually treated by: use water filter: no,water usually treated by: use water filter: yes,water usually treated by: use water filter: NaN,"owns livestock, herds or farm animals: no","owns livestock, herds or farm animals: yes","owns livestock, herds or farm animals: NaN"
0,AOGE52FL,Ambaca,Kuanza Norte,Angola,2007,3,2007-03-06,5.160000,5.0,1.907878,...,0.0,0.920000,0.080000,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
1,AOGE52FL,Amboim,Cuanza Sul,Angola,2006,12,2006-12-11,5.280000,5.0,2.282542,...,0.0,0.600000,0.400000,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
2,AOGE52FL,Andulo,Bie,Angola,2007,1,2007-01-08,4.000000,4.0,1.632993,...,0.0,0.920000,0.080000,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
3,AOGE52FL,Baia Farta,Benguela,Angola,2006,11,2006-11-27,5.208333,5.0,2.283764,...,0.0,0.791667,0.208333,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
4,AOGE52FL,Balombo,Benguela,Angola,2006,12,2006-12-07,4.210526,4.0,1.618605,...,0.0,0.947368,0.052632,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15247,ZWGE72FL,Umzingwane,Matabeleland South,Zimbabwe,2015,7,2015-07-10,4.642857,4.0,2.865006,...,0.0,0.976190,0.023810,0.0,0.97619,0.02381,0.000000,0.190476,0.809524,0.0
15248,ZWGE72FL,Uzumba-Maramba-Pfungwe,Mashonaland East,Zimbabwe,2015,10,2015-10-12,4.075949,4.0,2.080262,...,0.0,1.000000,0.000000,0.0,1.00000,0.00000,0.000000,0.151899,0.848101,0.0
15249,ZWGE72FL,Zaka,Masvingo,Zimbabwe,2015,10,2015-10-19,4.366337,4.0,2.415462,...,0.0,1.000000,0.000000,0.0,1.00000,0.00000,0.000000,0.069307,0.930693,0.0
15250,ZWGE72FL,Zvimba,Mashonaland West,Zimbabwe,2015,11,2015-11-25,4.546154,4.0,2.385718,...,0.0,0.976923,0.023077,0.0,1.00000,0.00000,0.007692,0.361538,0.638462,0.0


In [38]:
grouped_df.to_csv(out_f.replace('.pkl', '.csv'), index=False)
grouped_df.to_pickle(out_f)